In [1]:
// 使用须知
// 1. 用户需要有tmp表的写入权限，申请方法见：https://km.sankuai.com/page/411726480
// 2. 需要增加git权限，将你的公钥添加到https://dev.sankuai.com/code/home 又上角 SSH Key
// 3. 需要升级AFO的版本，（在提交机 /opt/meituan/versions 目录下 下载hdfs上的/user/hadoop-hdp/dushifeng/afo-clients-0.5.70.tar.gz 这个文件， ll /opt/meituan/versions/afo-clients-0.5.70/afo-tensorflow/afo-tensorflow-0.5.70/
// 然后执行 ln -snf /opt/meituan/versions/afo-clients-0.5.70/afo-tensorflow/afo-tensorflow-0.5.70 /opt/meituan/tensorflow-release 试一下
println("Start!!!")

Start!!!


In [2]:
// spark 初始化，为后续sql的运行准备环境
import $ivy.`com.alibaba:fastjson:1.2.70`
import com.alibaba.fastjson.{JSON, JSONObject}
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)
import org.apache.spark.sql._
import org.apache.spark.{SparkConf, SparkContext}
var user = sys.env.get("JUPYTERHUB_USER").get
val conf = {
   new SparkConf()
   .setAppName(s"Online-Feature-For-Bid-${user}")
   .setMaster("yarn")
   .set("spark.yarn.queue", "root.zw03.hadoop-waimaiadrd.etl") // spark 的队列，可以根据自己的需要设置
   .set("spark.yarn.archive", "viewfs://hadoop-meituan/user/hadoop-hdp/notebook/spark-jars/spark_libs.zip")
   .set("spark.yarn.stagingDir", "viewfs://hadoop-meituan/ghnn03/sparkstaging")
   .set("spark.executor.extraLibraryPath", "/opt/meituan/nodemanager/lib/native/:/opt/meituan/hadoop/lib/native/")
   .set("spark.driver.extraLibraryPath", "/opt/meituan/nodemanager/lib/native/:/opt/meituan/hadoop/lib/native/")
   .set("spark.executorEnv.JAVA_HOME", "'/usr/local/java18'")
   .set("spark.yarn.appMasterEnv.JAVA_HOME", "'/usr/local/java18'")
   .set("spark.sql.hive.metastorePartitionPruning", "true")
   .set("spark.shuffle.service.enabled", "true")
   .set("spark.dynamicAllocation.enabled", "true")
   .set("spark.dynamicAllocation.minExecutors", "600") //spark的执行Executors
   .set("spark.dynamicAllocation.maxExecutors","1000")
   .set("spark.executor.memory","16G")
   .set("spark.driver.memory","7G")
   .set("spark.input.dir.recursive","true")
   .set("spark.sql.ignore.existed.function.enable","true")
   .set("spark.yarn.maxAppAttempts", "2")
   .set("spark.noResourceMaxInterval","86400s")
   .set("spark.scheduler.mode", "FAIR")
   .set("spark.executor.cores","8")
   .set("spark.yarn.executor.memoryOverhead","32024")
   .set("spark.yarn.driver.memoryOverhead","32024")
   .set("spark.hadoop.hive.mt.renew.token.enable", "true")
}
val spark = {
 NotebookSparkSession
   .builder()
   .config(conf)
   .enableHiveSupport.getOrCreate
}
// 引入相关的库
import java.text.SimpleDateFormat
import java.util.Calendar
import ammonite.ops.ImplicitWd._
import ammonite.ops._
import sys.process._
import java.io._
spark.sparkContext.applicationId

var logo="AFO_RL"
// 引入相关的库
import java.text.SimpleDateFormat
import java.util.Calendar
import ammonite.ops.ImplicitWd._
import ammonite.ops._
import sys.process._
import java.io._
import java.text.SimpleDateFormat
import java.util.Date

SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#noProviders for further details.
SLF4J: Class path contains SLF4J bindings targeting slf4j-api versions prior to 1.8.
SLF4J: Ignoring binding found at [jar:file:/conda/envs/notebook/share/jupyter/kernels/scala211/pack/lib/slf4j-nop-1.7.26.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#ignoredBindings for an explanation.


Adding Hadoop conf dir /opt/meituan/hadoop/etc/hadoop to classpath
Adding Hive conf dir /opt/meituan/spark-2.2/conf to classpath
Creating SparkSession


log4j:WARN No appenders could be found for logger (io.netty.util.internal.logging.InternalLoggerFactory).
log4j:WARN Please initialize the log4j system properly.
log4j:WARN See http://logging.apache.org/log4j/1.2/faq.html#noconfig for more info.


Spark UI

import $ivy.$                            

import com.alibaba.fastjson.{JSON, JSONObject}

import org.apache.log4j.{Level, Logger}

import org.apache.spark.sql._

import org.apache.spark.{SparkConf, SparkContext}

user: String = "wanghao251"
conf: SparkConf = org.apache.spark.SparkConf@54cd8e96
spark: SparkSession = org.apache.spark.sql.SparkSession@1644fab2
import java.text.SimpleDateFormat

import java.util.Calendar

import ammonite.ops.ImplicitWd._

import ammonite.ops._

import sys.process._

import java.io._

res1_15: String = "application_1671689962742_1340598"
logo: String = "AFO_RL"
import java.text.SimpleDateFormat

import java.util.Calendar

import ammonite.ops.ImplicitWd._

import ammonite.ops._

import sys.process._

import java.io._

import java.text.SimpleDateFormat

import java.util.Date

In [3]:
// 通用udf注册
import scala.collection.mutable.ArrayBuffer
import scala.math
import scala.util.Random;
import java.lang.Math
import com.alibaba.fastjson.{JSON, JSONObject}

import org.apache.spark.sql.{DataFrame, Row}
import org.apache.spark.sql.types.{ArrayType, FloatType, DoubleType, IntegerType, LongType, StringType, StructField, StructType}

import scala.collection.immutable.Map
import scala.collection.mutable

def JSONObj2Map(jsonObj: JSONObject): Map[String, String] = {
    var map = mutable.Map[String, String]()
    if(!jsonObj.isEmpty){
        val itr: java.util.Iterator[String] = jsonObj.keySet().iterator()
        while (itr.hasNext) {
            val key = itr.next()
            map += ((key, jsonObj.getString(key)))
        }
    }
    map.toMap
}

import org.apache.hadoop.hive.ql.exec.UDF
import spark.implicits._
val findIndex2 =  (strQuote:String) => {
    val poi = strQuote.split("\073").last
    val ind = strQuote.split("\073").zipWithIndex.filter(_._1 == poi).map(_._2).min
    ind.toString
}
spark.sqlContext.udf.register("findIndex2", findIndex2)
spark.udf.register("findIndex2", findIndex2)

def getDate(delta:Int, current_day:String = "today"):String = {
    val sdf:SimpleDateFormat = new SimpleDateFormat("yyyyMMdd")
    val cal:Calendar = Calendar.getInstance()
    if ( !current_day.equals("today")){
      val dt = sdf.parse(current_day)
      cal.setTime(dt)
    }
    cal.add(Calendar.DATE,delta)
    val day = sdf.format(cal.getTime)
    day
}
def getTimeSuffix():String={
    val dateFormat = new SimpleDateFormat("yyyyMMdd")
    val cla = Calendar.getInstance()
    cla.setTimeInMillis(System.currentTimeMillis())
    val table_suffix = dateFormat.format(cla.getTime)
    table_suffix
}
def dropTable(sqlContext:SQLContext, tableName:String, is_execute_sql:Boolean):Unit={
    printCurTime()
    var sql = s"Drop table if exists ${tableName}"
    if(is_execute_sql){
        println(s"执行 \n ${sql}")
        sqlContext.sql(sql)
    }
    printCurTime()
}

def executeSQL(sqlContext:SQLContext, sql:String, is_execute_sql:Boolean):DataFrame={
    printCurTime()
    println(s"执行 \n ${sql}")
    var result_ = sqlContext.sql(sql)
    printCurTime()
    result_
}

def genDateList(start: String, end: String): List[String] = {
    import java.util.Date
    import org.joda.time.{DateTime, Instant, Days}
    import org.joda.time.format.{DateTimeFormat, DateTimeFormatter}

    val DATE_KEY_FORMAT: DateTimeFormatter = DateTimeFormat.forPattern("yyyyMMdd")
    val DATE_FORMAT: DateTimeFormatter = DateTimeFormat.forPattern("yyyyMMdd")

    val startDate = DATE_KEY_FORMAT.parseDateTime(start).toDateTime
    val endDate = DATE_KEY_FORMAT.parseDateTime(end).toDateTime

    val daysCount = Days.daysBetween(startDate, endDate).getDays() + 1
    (0 until daysCount).map(startDate.plusDays(_)).map(d => d.toString(DATE_FORMAT)).toList
  }



def printCurTime():Unit={
    println(new SimpleDateFormat("yyyy-MM-dd HH:mm:ss").format(new Date))
}


def showTopNFromSQL(sql:String, n:Int):Unit={
    printCurTime()
    var result = executeSQL(spark.sqlContext,sql,true)
    result.show(n)
    printCurTime()
}
def executeSQLLazy(sql:String):DataFrame={
    executeSQL(spark.sqlContext,sql,true)
}
def showTopNFromTableName(tableName:String, n:Int):Unit={
    printCurTime()
    var result = executeSQL(spark.sqlContext,s"""select * from ${tableName} limit ${n}""",true)
    result.show(n)
    printCurTime()
}
def executeShell(shell:String):Unit={
    printCurTime()
    println("----------------The content of shell is: -------------------")
    println(shell)
    println("----------------Start execution....-------------------")
    var file_name=System.currentTimeMillis()
    val writer = new PrintWriter(new File(s"/opt/meituan/dolphinfs_${user}/${file_name}.sh" ))
    writer.write(shell)
    writer.close()
    //启动脚本
    s"chmod +777 /opt/meituan/dolphinfs_${user}/${file_name}.sh".!
    s"sh /opt/meituan/dolphinfs_${user}/${file_name}.sh".!
    s"rm -r /opt/meituan/dolphinfs_${user}/${file_name}.sh".!
    println("----------------End-------------------")
    printCurTime()
}

def getFeatureMap(x:((String,String))):String={
    var x_1 = x._1
    var x_2 = x._2.replace("Type","")
    if (x_2.contains("Array")){
            var x_type = x_2.split('(')(1).split(",")(0)
            x_2  = s"""Array<${x_type}>"""
    }
    var x_3 = s"""${x_1} ${x_2}"""
    x_3
}
// 传入sql，返回结果表
def createTableFromSQLWithName(sql:String,tableName:String,database:String):Unit={
    var rowData=executeSQLLazy(sql)
    var tmpTableName=s"tmp_${System.currentTimeMillis()}"
    var finalTableName=database+"."+tableName
    var table_scheme = rowData.dtypes
    var columnsName = table_scheme.map{x=> x._1}.mkString(",")
    
    
    //var featureForCreateTable=table_scheme.map{x=> s"""${x._1} ${x._2.stripSuffix("Type")}""".replace("Type","")}.mkString(",")
    var featureForCreateTable=table_scheme.map(getFeatureMap).mkString(",")
    executeSQLLazy(s"""drop table if exists ${finalTableName}""")
    executeSQLLazy(s"""create table if not exists ${finalTableName}(${featureForCreateTable}) STORED AS ORC""")
    rowData.createOrReplaceTempView(tmpTableName)
    executeSQLLazy(s"insert OVERWRITE table ${finalTableName} select ${columnsName} from ${tmpTableName}")
}

def createTableFromSQL(sql:String):String={
    var finalTableName=s"${logo}_${System.currentTimeMillis()}"
    createTableFromSQLWithName(sql,finalTableName,"tmp")
    "tmp."+finalTableName
}
def createTableFromSQL(sql:String,database:String):String={
    var finalTableName=s"${logo}_${System.currentTimeMillis()}"
    createTableFromSQLWithName(sql,finalTableName,database)
    database+"."+finalTableName
}

def generateTrajectory(tableName:String, groupColumns:Array[String], rankColumns:Array[String], joinColumns:Array[String],database:String):String={
    
    println(tableName.split("."))
    var finalTableName=database+"."+tableName.split("\\.")(1)+"_trajectory"
    
    var table_scheme = executeSQLLazy(s"""select * from ${tableName} limit 1""").dtypes
    var columnsName = table_scheme.map{x=> x._1}.mkString(",")
    var groupColumnString=groupColumns.mkString(",")
    var rankColumnString = rankColumns.mkString(",")
    var joinConditionString = joinColumns.map(x=> s"t1.${x}=t2.${x} ").mkString("\n and ")
    var selectT1columnsNameString = table_scheme.map{x=> s"t1.${x._1}"}.mkString(",")
    var selectT2columnsNameString = table_scheme.map{x=> s"t2.${x._1} as next_${x._1}"}.mkString(",")
    //var featureForCreateTable=table_scheme.map(getFeatureMap).mkString(",")
    var featureForCreateTable=table_scheme.map(getFeatureMap).mkString(",") + ",rn int,"+table_scheme.map(x=> {
        var x_1 = x._1
        var x_2 = x._2.replace("Type","")
        if (x_2.contains("Array")){
            var x_type = x_2.split('(')(1).split(",")(0)
            x_2  = s"""Array<${x_type}>"""
        }
        
        s"""next_${x_1} ${x_2}"""}
                                              ).mkString(",")+",next_rn int"
    var allColumnString=table_scheme.map{x=> s"${x._1}"}.mkString(",")+",rn,"+table_scheme.map{x=> s"next_${x._1}"}.mkString(",")+",next_rn"
    var sql=s"""
        select ${selectT1columnsNameString},t1.rn,${selectT2columnsNameString},t2.rn as next_rn
        from(
            select ${columnsName}, RANK() OVER(PARTITION BY ${groupColumnString} ORDER BY ${rankColumnString} asc) AS rn 
            from ${tableName}
        )t1
        left join(
            select ${columnsName}, RANK() OVER(PARTITION BY ${groupColumnString} ORDER BY ${rankColumnString} asc) AS rn 
            from ${tableName}
        )t2
        on t1.rn+1 = t2.rn
        and ${joinConditionString}
        """
    var trajectoryData=executeSQLLazy(sql)
    var tmpTableName=s"tmp_${System.currentTimeMillis()}"
    executeSQLLazy(s"""drop table if exists ${finalTableName}""")
    executeSQLLazy(s"""create table if not exists ${finalTableName}(${featureForCreateTable}) STORED AS ORC""")
    trajectoryData.createOrReplaceTempView(tmpTableName)
    spark.sql(s"insert OVERWRITE table ${finalTableName} select ${allColumnString} from ${tmpTableName}")
    finalTableName
}
def createPTTableFromTable(tableName:String,targetTableName:String,database:String):String={
    var finalTableName=targetTableName
    var rowData=executeSQLLazy(s"select * from ${tableName} limit 1")
    var tmpTableName=s"${database}.${logo}_${System.currentTimeMillis()}"
    var table_scheme = rowData.dtypes
    var columnsName = table_scheme.map{x=> x._1}.mkString(",")
    var featureForCreateTable=table_scheme.map(getFeatureMap).mkString(",")
    executeSQLLazy(s"""drop table if exists ${finalTableName}""")
    executeSQLLazy(s"""create table if not exists ${finalTableName}(${featureForCreateTable})  PARTITIONED BY (pt string) STORED AS ORC""")
    rowData.createOrReplaceTempView(tmpTableName)
    var dt = "today"
    var date_s=getDate(-1,dt)
    spark.sql(s"INSERT OVERWRITE TABLE ${finalTableName} PARTITION (pt='${date_s}') select ${columnsName} from ${tmpTableName}")
    finalTableName
}

def createPTTableFromTableWithSpace(tableName:String,database:String):String={
    var finalTableName=database+s".${logo}_${System.currentTimeMillis()}"
    var rowData=executeSQLLazy(s"select * from ${tableName}")
    var tmpTableName=s"tmp_${System.currentTimeMillis()}"
    var table_scheme = rowData.dtypes
    var columnsName = table_scheme.map{x=> x._1}.mkString(",")
    var featureForCreateTable=table_scheme.map(getFeatureMap).mkString(",")
    executeSQLLazy(s"""drop table if exists ${finalTableName}""")
    executeSQLLazy(s"""create table if not exists ${finalTableName}(${featureForCreateTable})  PARTITIONED BY (pt string) STORED AS ORC""")
    rowData.createOrReplaceTempView(tmpTableName)
    var dt = "today"
    var date_s=getDate(-1,dt)
    spark.sql(s"INSERT OVERWRITE TABLE ${finalTableName} PARTITION (pt=${date_s}) select ${columnsName} from ${tmpTableName}")
    finalTableName
}

import scala.collection.mutable.ArrayBuffer

import scala.math

import scala.util.Random;

import java.lang.Math

import com.alibaba.fastjson.{JSON, JSONObject}


import org.apache.spark.sql.{DataFrame, Row}

import org.apache.spark.sql.types.{ArrayType, FloatType, DoubleType, IntegerType, LongType, StringType, StructField, StructType}


import scala.collection.immutable.Map

import scala.collection.mutable


defined function JSONObj2Map
import org.apache.hadoop.hive.ql.exec.UDF

import spark.implicits._

findIndex2: String => String = <function1>
res2_13: expressions.UserDefinedFunction = UserDefinedFunction(
  <function1>,
  StringType,
  Some(List(StringType))
)
res2_14: expressions.UserDefinedFunction = UserDefinedFunction(
  <function1>,
  StringType,
  Some(List(StringType))
)
defined function getDate
defined function getTimeSuffix
defined function dropTable
defined function executeSQL
defined function genDateList
defined function printCurTime
defined function showTopNFromSQL
def

In [4]:
// 配置区
val is_execute_sql = true
var table_suffix = "20230102"
var befor1day = "20230102"
var fp_log_date_start="20230102"
var fp_log_date_end="20230102"
var predict_dt="20230102"
var business="dsa_os_unify_model" //提取特征的业务线

is_execute_sql: Boolean = true
table_suffix: String = "20230102"
befor1day: String = "20230102"
fp_log_date_start: String = "20230102"
fp_log_date_end: String = "20230102"
predict_dt: String = "20230102"
business: String = "dsa_os_unify_model"

In [5]:
// 获取featureID 和名字的对应关系
var featureId2Name=s"""
select distinct id, name
from origindb_ss.waimaiadtcpc_waimai_offline_feature_platform__wm_model_feature_map
where dt=${befor1day}
and status=0
"""
//构建 id到name映射的map
val featureId2NameMap = spark.sqlContext.sql(featureId2Name).na.drop().rdd.
      map(f => (f.getLong(0).toString, f.getString(1))).
      collectAsMap()


val feaArray = Array(
    "pvid string",
    "user_id bigint",
    "uuid string",
    "poi_id bigint",
    "request_time bigint",
    "qsbid double",
    "precisebid double",
    "demandbiztype bigint",
    "supplybiztype bigint",
    "is_expose bigint",
    "is_click bigint",
    "final_charge double",
    "is_order bigint",
    "revenue double",
    "rt_features string",
    "raw_features string",
    "modelname string",
    "business string",
    "params string",
    "dt string"
)

var logRawFeature=""

// 构建schema
import scala.collection.mutable.ArrayBuffer
val schemaArray = new ArrayBuffer[StructField]()
var columeNameSet= new mutable.HashSet[String]()
for (kv <- feaArray.map(_.split(" "))) {
    val groupKey = kv.head
    val groupType = kv.last
    columeNameSet.add(groupKey)
    logRawFeature+=groupKey+","
    if (groupType.startsWith("double")) {
        schemaArray += StructField(groupKey, DoubleType)
    } else if (groupType.startsWith("int")) {
        schemaArray += StructField(groupKey, IntegerType)
    } else if (groupType.startsWith("bigint")) {
        schemaArray += StructField(groupKey, LongType)
    } else {
        schemaArray += StructField(groupKey, StringType)
    }
}
logRawFeature=logRawFeature.stripSuffix(",")


var line=executeSQLLazy(s"""
select rt_features,raw_features,biddetails
from mart_waimaiad.dsa_model_fp_os_rl_log_wh
where dt = "${fp_log_date_end}"
limit 1
""")
//scame 构建
var rt_map:Map[String, String] = Map()
var rawFeatureMap:Map[String, String] = Map()



line.collect().foreach(x=>{
    var rt_features=x.getString(0)
    var rawFeatureString= x.getString(1)
    var detailsString=x.getString(2)
    println("before")
    println(rt_features)
    println(rawFeatureString)
    var rtMap =  JSONObj2Map(JSON.parseObject(rt_features))
    var rawFeature =  JSONObj2Map(JSON.parseObject(rawFeatureString))
    var detail=JSONObj2Map(JSON.parseObject(detailsString)).get("preciseBidFinalDetailMap").get
    println(detail)
    for(key<-rtMap.keys){
        //类型增加判断 todo:
        if(!columeNameSet.contains(featureId2NameMap.getOrElse(key, key))){
            //println(s"${key}:${rtMap.get(key).get}, ${rtMap.get(key).get.getClass.getSimpleName}")
            schemaArray+= StructField(featureId2NameMap.getOrElse(key, key), StringType)
            rt_map+=(key->featureId2NameMap.getOrElse(key, key))
            columeNameSet.add(featureId2NameMap.getOrElse(key, key))
        }
    }
    for(key<-rawFeature.keys){
        if(!columeNameSet.contains(key)){
            // println(s"${key}:${rawFeature.get(key).get}, ${rawFeature.get(key).get.getClass.getSimpleName}")
            schemaArray+= StructField(key, StringType)
            rawFeatureMap+=(key->key)
            columeNameSet.add(featureId2NameMap.getOrElse(key, key))
        }
    }
    
})

var detail_feature_list = Array("rlkAct", "delete_res")
for (key <-detail_feature_list){
    schemaArray+= StructField(key, StringType)
    columeNameSet.add(key)
}
println("after")
println(rt_map)
println(rawFeatureMap)

rdd at cmd4.sc:8

1 / 1

collectAsMap at cmd4.sc:10

200 / 200

2023-01-04 17:49:59
执行 
 
select rt_features,raw_features,biddetails
from mart_waimaiad.dsa_model_fp_os_rl_log_wh
where dt = "20230102"
limit 1

2023-01-04 17:49:59


collect at cmd4.sc:72

1 / 1

before
{"92310":294.0,"68503":12.0,"146102":0.0,"8531":17.0,"286":0.0}
{"histctr":0.03724542988969873,"winningscore_comp12":0.0,"real_bln":0.0,"order_num_before1week":0.0,"real_csm_bln":0.01,"upperk":1.2,"charge_before1week":0.0,"revenue_before4week":384.2099914550781,"real_budget":30000.0,"impr_before3week":24.0,"revenue":100.70999908447266,"cpc_before3week":0.6399999856948853,"lowerk":0.9,"impr":181.0,"cpc":0.8291666507720947,"real_csm":0.0,"impr_before4week":1245.0,"ctr":0.05447192005813122,"click_before2week":97.0,"revenue_before1week":0.0,"mt_butie_before1week":0.0,"impr_before1week":0.0,"revenue_before2week":604.8200073242188,"order_num_before3week":1.0,"cityid":310118.0,"click_before1week":0.0,"mt_butie":5.71999979019165,"aimcpc":90.0,"real_cpc":0.0,"feacomp_ctr":1.2724618358140183,"ratioctr_exp":1.0997017676287786,"revenue_before3week":24.799999237060547,"mt_butie_before2week":62.84000015258789,"order_num":2.0,"order_num_before2week":18.0,"cpc_before4week":0.554411768913269,"im

featureId2Name: String = """
select distinct id, name
from origindb_ss.waimaiadtcpc_waimai_offline_feature_platform__wm_model_feature_map
where dt=20230102
and status=0
"""
featureId2NameMap: collection.Map[String, String] = Map(
  "152664" -> "thh_main_hot_gmv_poi_order_cnt_3day",
  "68746" -> "sg_b2c_product_comment_cnt_30day",
  "128635" -> "thh_whole_scene_uuid_prod_second_cate_order_cnt_90day",
  "8946" -> "query_poi_max_query_term_poi_vooeo_7d",
  "16136" -> "poi_franchise_poislot_30_90_pv",
  "146004" -> "sg_search_stg_user_view_poi_count_30d",
  "68440" -> "sg_search_query_spuname_bert_semantic_relevance_prob",
  "191468" -> "cpc_ad_nl_req_avg_ctr",
  "165028" -> "thh_poi_all_scene_view_cnt_3day",
  "93546" -> "sg_search_rt_spu_all_discount_amount_l1_origin",
  "10481" -> "rel_food_count",
  "76054" -> "query_click_top20_poi_list_90day",
  "590" -> "ka_order_90days",
  "77787" -> "hoae_cvr_tagid_order_count_30_v1",
  "10667" -> "list_click_poi_embedding_cur",
  "157435" -> "log

In [6]:
//将log表的特征列展开
var tmpFPExtendRTFeatureTableName=s"${user}_${business}_${System.currentTimeMillis()}"
var FPExtendRTFeaturetableName=s"tmp.dsa_model_fp_log_${user}_${business}_df_${fp_log_date_end}_high"

var logData=s"""
select ${logRawFeature},biddetails
from mart_waimaiad.dsa_model_fp_os_rl_log_wh
where dt between ${fp_log_date_start} and ${fp_log_date_end}
and business ="${business}"
"""
println(logData)

var rawData = spark.sqlContext.sql(logData).rdd.flatMap{
      f =>
        var pvid=f.getAs[String]("pvid")
        var user_id=f.getAs[Long]("user_id")
        var uuid=f.getAs[String]("uuid")
        var poi_id=f.getAs[Long]("poi_id")
        var request_time=f.getAs[Long]("request_time")
        var rawbid=f.getAs[Double]("qsbid")
        var precisebid=f.getAs[Double]("precisebid")
        var demandbiztype=f.getAs[Long]("demandbiztype")
        var supplybiztype=f.getAs[Long]("supplybiztype")
        var is_expose=f.getAs[Long]("is_expose")
        var is_click=f.getAs[Long]("is_click")
        var final_charge=f.getAs[Double]("final_charge")
        var is_order=f.getAs[Long]("is_order")
        var revenue=f.getAs[Double]("revenue")
        var rt_features=f.getAs[String]("rt_features").replace("Infinity","-1.0")
        var raw_features=f.getAs[String]("raw_features").replace("Infinity","-1.0")
        var detail_features =f.getAs[String]("biddetails").replace("Infinity","-1.0")
        var modelname=f.getAs[String]("modelname")
        var business=f.getAs[String]("business")
        var params=f.getAs[String]("params")
        var dt = f.getAs[String]("dt")
        var array =Array(pvid,user_id,uuid,poi_id,request_time,rawbid,precisebid,demandbiztype,supplybiztype,is_expose,is_click,final_charge,is_order,revenue,rt_features,raw_features,modelname,business,params,dt)
        var rtFeaturesMap = JSONObj2Map(JSON.parseObject(rt_features))
        var rawFeaturesLawMap = JSONObj2Map(JSON.parseObject(raw_features))
        var preciseBidFinalDetailMap = JSONObj2Map(JSON.parseObject(detail_features)).getOrElse("preciseBidFinalDetailMap","{}")
        var detail=JSONObj2Map(JSON.parseObject(preciseBidFinalDetailMap))
        //当前只支持int，string和double的类型，复杂类型还未支持
        for(featureID<-rt_map.keys){
            if(rtFeaturesMap.contains(featureID)){
                array=array:+rtFeaturesMap.get(featureID).get
            }else{
                array=array:+ -0.678
            }
        }
        for(featureID<-rawFeatureMap.keys){
            if(rawFeatureMap.contains(featureID)){
                array=array:+rawFeaturesLawMap.get(featureID).get
            }else{
                array=array:+"unknown"
            }
        }
        
        for(featureID<-detail_feature_list){
            if(detail.contains(featureID)){
                array=array:+detail.get(featureID).get
            }else{
                array=array:+ "unknown"
            }
        }
        
        var row =Row(array:_*)
        // var row = Row(pvid)
        List(row)
}
println(rawData.count())
println("test")


select pvid,user_id,uuid,poi_id,request_time,qsbid,precisebid,demandbiztype,supplybiztype,is_expose,is_click,final_charge,is_order,revenue,rt_features,raw_features,modelname,business,params,dt,biddetails
from mart_waimaiad.dsa_model_fp_os_rl_log_wh
where dt between 20230102 and 20230102
and business ="dsa_os_unify_model"



count at cmd5.sc:68

2176 / 2000

857434405
test


tmpFPExtendRTFeatureTableName: String = "wanghao251_dsa_os_unify_model_1672825803850"
FPExtendRTFeaturetableName: String = "tmp.dsa_model_fp_log_wanghao251_dsa_os_unify_model_df_20230102"
logData: String = """
select pvid,user_id,uuid,poi_id,request_time,qsbid,precisebid,demandbiztype,supplybiztype,is_expose,is_click,final_charge,is_order,revenue,rt_features,raw_features,modelname,business,params,dt,biddetails
from mart_waimaiad.dsa_model_fp_os_rl_log_wh
where dt between 20230102 and 20230102
and business ="dsa_os_unify_model"
"""
rawData: org.apache.spark.rdd.RDD[Row] = MapPartitionsRDD[26] at flatMap at cmd5.sc:12

In [7]:
// 将数据写入到hive表
val tmpTable = spark.sqlContext.createDataFrame(rawData, StructType(schemaArray.toList))
tmpTable.toDF().registerTempTable(tmpFPExtendRTFeatureTableName)
var featureForCreateTable=s""
var allFeatures=""
for(i <- 0 to schemaArray.length-1){
    allFeatures=allFeatures+schemaArray(i).name+","
    if(i==schemaArray.length-1){
        featureForCreateTable=featureForCreateTable+schemaArray(i).name+" "+schemaArray(i).dataType.typeName
    }else{
        featureForCreateTable=featureForCreateTable+schemaArray(i).name+" "+schemaArray(i).dataType.typeName+","
    }
}
allFeatures=allFeatures.stripSuffix(",")
executeSQLLazy(s"""drop table if exists $FPExtendRTFeaturetableName""")
executeSQLLazy(s"""create table if not exists $FPExtendRTFeaturetableName(${featureForCreateTable}) STORED AS ORC""")
executeSQLLazy(s"""insert overwrite table ${FPExtendRTFeaturetableName} select ${allFeatures} from ${tmpFPExtendRTFeatureTableName} where cast(ctr as float)+cast(histctr as float)+cast(final_charge as float)+cast(real_csm as float)+cast(aimcpc as float)+cast(impr as float)+cast(cpc as float)+cast(revenue as float)+cast(charge as float)+cast(click as float)+cast(order_num as float)+cast(mt_butie as float)+cast(impr_before1week as float)+cast(click_before1week as float)+cast(cpc_before1week as float)+cast(order_num_before1week as float)+cast(revenue_before1week as float)+cast(mt_butie_before1week as float)+cast(charge_before1week as float)+cast(impr_before2week as float)+cast(click_before2week as float)+cast(cpc_before2week as float)+cast(order_num_before2week as float)+cast(revenue_before2week as float)+cast(mt_butie_before2week as float)+cast(charge_before2week as float)+cast(impr_before3week as float)+cast(click_before3week as float)+cast(cpc_before3week as float)+cast(order_num_before3week as float)+cast(revenue_before3week as float)+cast(mt_butie_before3week as float)+cast(charge_before3week as float)+cast(impr_before4week as float)+cast(click_before4week as float)+cast(cpc_before4week as float)+cast(order_num_before4week as float)+cast(revenue_before4week as float)+cast(mt_butie_before4week as float)+cast(charge_before4week as float) < 1E6
and   delete_res!='1.01' and delete_res!='1.04' """)
// showTopNFromTableName(tmpFPExtendRTFeatureTableName,10)

2023-01-04 17:55:41
执行 
 drop table if exists tmp.dsa_model_fp_log_wanghao251_dsa_os_unify_model_df_20230102
2023-01-04 17:55:41
2023-01-04 17:55:41
执行 
 create table if not exists tmp.dsa_model_fp_log_wanghao251_dsa_os_unify_model_df_20230102(pvid string,user_id long,uuid string,poi_id long,request_time long,qsbid double,precisebid double,demandbiztype long,supplybiztype long,is_expose long,is_click long,final_charge double,is_order long,revenue double,rt_features string,raw_features string,modelname string,business string,params string,dt string,unused string,click string,user_tag_pref_click_3day string,hour string,user_ord_num_weekdays string,charge_before2week string,order_num_before2week string,impr_before1week string,feacomp_cvr string,click_before1week string,lowerk string,real_cpc string,real_csm string,revenue_before3week string,cpc_before1week string,charge_before3week string,winningscore_comp12 string,impr string,real_cst_bdgt string,real_aBln string,feacomp_ctr string,real_

sql at cmd2.sc:66

2000 / 2000

Moved: 'viewfs://hadoop-meituan/user/hive/warehouse/tmp.db/dsa_model_fp_log_wanghao251_dsa_os_unify_model_df_20230102' to trash at: hdfs://dfsrouter/user/hadoop-hmart-waimaiad/.Trash/Current
2023-01-04 18:19:33


tmpTable: DataFrame = [pvid: string, user_id: bigint ... 88 more fields]
featureForCreateTable: String = "pvid string,user_id long,uuid string,poi_id long,request_time long,qsbid double,precisebid double,demandbiztype long,supplybiztype long,is_expose long,is_click long,final_charge double,is_order long,revenue double,rt_features string,raw_features string,modelname string,business string,params string,dt string,unused string,click string,user_tag_pref_click_3day string,hour string,user_ord_num_weekdays string,charge_before2week string,order_num_before2week string,impr_before1week string,feacomp_cvr string,click_before1week string,lowerk string,real_cpc string,real_csm string,revenue_before3week string,cpc_before1week string,charge_before3week string,winningscore_comp12 string,impr string,real_cst_bdgt string,real_aBln string,feacomp_ctr string,real_expose string,real_revenue string,revenue_before2week string,ratioctr string,real_price string,lowerbid string,ratio_aimcpc string,cpc_bef

In [8]:

var processTable_high_min_max = createTableFromSQL(s"""
SELECT a.poi_id,
       a.dt,
       a.prod,
       begin.real_budget as begin_budget,
       begin.real_bln as begin_bln,
       end.real_bln as end_bln,
       end.real_budget as end_budget
  FROM(
        SELECT poi_id,
               dt,
               prod,
               min(request_time) as budget_time,
               max(request_time) as goal_time
          FROM ${FPExtendRTFeaturetableName}
         GROUP BY poi_id,
                  dt,
                  prod
       )a
  LEFT JOIN (
        SELECT poi_id,
               dt,
               prod,
               real_bln,
               real_budget,
               request_time
          from ${FPExtendRTFeaturetableName}
       )begin
    ON a.poi_id = begin.poi_id
       and a.dt = begin.dt
       and a.prod = begin.prod
       and a.budget_time = begin.request_time
  LEFT JOIN (
        SELECT poi_id,
               dt,
               prod,
               real_bln,
               real_budget,
               request_time
          from ${FPExtendRTFeaturetableName}
       )end
    ON a.poi_id = end.poi_id
       and a.dt = end.dt
       and a.prod = end.prod
       and a.goal_time = end.request_time
""")

2023-01-04 18:19:34
执行 
 
SELECT a.poi_id,
       a.dt,
       a.prod,
       begin.real_budget as begin_budget,
       begin.real_bln as begin_bln,
       end.real_bln as end_bln,
       end.real_budget as end_budget
  FROM(
        SELECT poi_id,
               dt,
               prod,
               min(request_time) as budget_time,
               max(request_time) as goal_time
          FROM tmp.dsa_model_fp_log_wanghao251_dsa_os_unify_model_df_20230102
         GROUP BY poi_id,
                  dt,
                  prod
       )a
  LEFT JOIN (
        SELECT poi_id,
               dt,
               prod,
               real_bln,
               real_budget,
               request_time
          from tmp.dsa_model_fp_log_wanghao251_dsa_os_unify_model_df_20230102
       )begin
    ON a.poi_id = begin.poi_id
       and a.dt = begin.dt
       and a.prod = begin.prod
       and a.budget_time = begin.request_time
  LEFT JOIN (
        SELECT poi_id,
               dt,
               p

sql at cmd2.sc:66

2000 / 2000

sql at cmd2.sc:66

2000 / 2000

...

[Stage 8:======================================================>(199 + 1) / 200]

sql at cmd2.sc:66

200 / 200

Moved: 'viewfs://hadoop-meituan/user/hive/warehouse/tmp.db/afo_rl_1672827574322' to trash at: hdfs://dfsrouter/user/hadoop-hmart-waimaiad/.Trash/Current
2023-01-04 18:28:28


processTable_high_min_max: String = "tmp.AFO_RL_1672827574322"

In [9]:

var processTable_high = createTableFromSQL(s"""
SELECT
cast(full_log.dt as String) dt,
cast(full_log.poi_id as String) as poi_id,
cast(full_log.prod as float) as prod,

-- cast(max(user_id) as string) as user_id,
-- cast(max(uuid) as string) as uuid,
cast(max(cityid) as float) as cityid,

cast(max(ctr) as float) as ctr,
cast(max(histctr) as float) as histctr,

-- 商家的goal，最终计费和cpc约束
cast(sum(final_charge) as float) as final_charge,
cast(max(real_csm) as float) as real_csm,

cast(max(aimcpc) as float) as aimcpc,

-- 过去一天，过去同期四周
cast(max(impr) as float) as impr,
cast(max(cpc) as float) as cpc,
cast(max(revenue) as float) as revenue,
cast(max(charge) as float) as charge,
cast(max(click) as float) as click,
cast(max(order_num) as float) as order_num,
cast(max(mt_butie) as float) as mt_butie,

cast(max(impr_before1week) as float) as impr_before1week,
cast(max(click_before1week) as float) as click_before1week,
cast(max(cpc_before1week) as float) as cpc_before1week,
cast(max(order_num_before1week) as float) as order_num_before1week,
cast(max(revenue_before1week) as float) as revenue_before1week,
cast(max(mt_butie_before1week) as float) as mt_butie_before1week,
cast(max(charge_before1week) as float) as charge_before1week,

cast(max(impr_before2week) as float) as impr_before2week,
cast(max(click_before2week) as float) as click_before2week,
cast(max(cpc_before2week) as float) as cpc_before2week,
cast(max(revenue_before2week) as float) as revenue_before2week,
cast(max(charge_before2week) as float) as charge_before2week,
cast(max(order_num_before2week) as float) as order_num_before2week,
cast(max(mt_butie_before2week) as float) as mt_butie_before2week,

cast(max(impr_before3week) as float) as impr_before3week,
cast(max(cpc_before3week) as float) as cpc_before3week,
cast(max(click_before3week) as float) as click_before3week,
cast(max(order_num_before3week) as float) as order_num_before3week,
cast(max(mt_butie_before3week) as float) as mt_butie_before3week,
cast(max(revenue_before3week) as float) as revenue_before3week,
cast(max(charge_before3week) as float) as charge_before3week,

cast(max(impr_before4week) as float) as impr_before4week,
cast(max(charge_before4week) as float) as charge_before4week,
cast(max(revenue_before4week) as float) as revenue_before4week,
cast(max(mt_butie_before4week) as float) as mt_butie_before4week,
cast(max(cpc_before4week) as float) as cpc_before4week,
cast(max(click_before4week) as float) as click_before4week,
cast(max(order_num_before4week) as float) as order_num_before4week,

cast(max(min_max_log.begin_budget) as float) as budget,

cast(sum(is_click) as float) as click_nums

FROM ${FPExtendRTFeaturetableName} 

as full_log
LEFT JOIN(
    SELECT * FROM ${processTable_high_min_max}) min_max_log 
    on full_log.poi_id = min_max_log.poi_id
    and full_log.dt = min_max_log.dt
    and full_log.prod = min_max_log.prod
GROUP BY full_log.dt,
full_log.poi_id,
full_log.prod
""")

2023-01-04 18:28:28
执行 
 
SELECT
cast(full_log.dt as String) dt,
cast(full_log.poi_id as String) as poi_id,
cast(full_log.prod as float) as prod,

-- cast(max(user_id) as string) as user_id,
-- cast(max(uuid) as string) as uuid,
cast(max(cityid) as float) as cityid,

cast(max(ctr) as float) as ctr,
cast(max(histctr) as float) as histctr,

-- 商家的goal，最终计费和cpc约束
cast(sum(final_charge) as float) as final_charge,
cast(max(real_csm) as float) as real_csm,

cast(max(aimcpc) as float) as aimcpc,

-- 过去一天，过去同期四周
cast(max(impr) as float) as impr,
cast(max(cpc) as float) as cpc,
cast(max(revenue) as float) as revenue,
cast(max(charge) as float) as charge,
cast(max(click) as float) as click,
cast(max(order_num) as float) as order_num,
cast(max(mt_butie) as float) as mt_butie,

cast(max(impr_before1week) as float) as impr_before1week,
cast(max(click_before1week) as float) as click_before1week,
cast(max(cpc_before1week) as float) as cpc_before1week,
cast(max(order_num_before1week) as float) as orde

run at ThreadPoolExecutor.java:1142

200 / 200

sql at cmd2.sc:66

2022 / 2000

[Stage 12:===================================================>(1999 + 1) / 2000]

sql at cmd2.sc:66

200 / 200

Moved: 'viewfs://hadoop-meituan/user/hive/warehouse/tmp.db/afo_rl_1672828108585' to trash at: hdfs://dfsrouter/user/hadoop-hmart-waimaiad/.Trash/Current
2023-01-04 18:30:47


processTable_high: String = "tmp.AFO_RL_1672828108585"

In [19]:
// 由于含有array，手动生产
// high_level

var tableNameAfterTrajectory_high=createTableFromSQL(s"""
SELECT t1.dt,
t1.poi_id,
array(sum(t1.ctr_dj),sum(t1.ctr_bj),sum(t1.ctr_ss),sum(t1.ctr_push)) as ctr,
array(sum(t1.histctr_dj),sum(t1.histctr_bj),sum(t1.histctr_ss),sum(t1.histctr_push)) as histctr,
array(sum(t1.aimcpc_dj),sum(t1.aimcpc_bj),sum(t1.aimcpc_ss),sum(t1.aimcpc_push)) as aimcpc,
array(sum(t1.impr_dj),sum(t1.impr_bj),sum(t1.impr_ss),sum(t1.impr_push)) as impr,
array(sum(t1.cpc_dj),sum(t1.cpc_bj),sum(t1.cpc_ss),sum(t1.cpc_push)) as cpc,
array(sum(t1.revenue_dj),sum(t1.revenue_bj),sum(t1.revenue_ss),sum(t1.revenue_push)) as revenue,
array(sum(t1.order_num_dj),sum(t1.order_num_bj),sum(t1.order_num_ss),sum(t1.order_num_push)) as order_num,
array(sum(t1.mt_butie_dj),sum(t1.mt_butie_bj),sum(t1.mt_butie_ss),sum(t1.mt_butie_push)) as mt_butie,
array(sum(t1.impr_before1week_dj),sum(t1.impr_before1week_bj),sum(t1.impr_before1week_ss),sum(t1.impr_before1week_push)) as impr_before1week,
array(sum(t1.click_before1week_dj),sum(t1.click_before1week_bj),sum(t1.click_before1week_ss),sum(t1.click_before1week_push)) as click_before1week,
array(sum(t1.cpc_before1week_dj),sum(t1.cpc_before1week_bj),sum(t1.cpc_before1week_ss),sum(t1.cpc_before1week_push)) as cpc_before1week,
array(sum(t1.order_num_before1week_dj),sum(t1.order_num_before1week_bj),sum(t1.order_num_before1week_ss),sum(t1.order_num_before1week_push)) as order_num_before1week,
array(sum(t1.revenue_before1week_dj),sum(t1.revenue_before1week_bj),sum(t1.revenue_before1week_ss),sum(t1.revenue_before1week_push)) as revenue_before1week,
array(sum(t1.mt_butie_before1week_dj),sum(t1.mt_butie_before1week_bj),sum(t1.mt_butie_before1week_ss),sum(t1.mt_butie_before1week_push)) as mt_butie_before1week,
array(sum(t1.charge_before1week_dj),sum(t1.charge_before1week_bj),sum(t1.charge_before1week_ss),sum(t1.charge_before1week_push)) as charge_before1week,
array(sum(t1.impr_before2week_dj),sum(t1.impr_before2week_bj),sum(t1.impr_before2week_ss),sum(t1.impr_before2week_push)) as impr_before2week,
array(sum(t1.click_before2week_dj),sum(t1.click_before2week_bj),sum(t1.click_before2week_ss),sum(t1.click_before2week_push)) as click_before2week,
array(sum(t1.cpc_before2week_dj),sum(t1.cpc_before2week_bj),sum(t1.cpc_before2week_ss),sum(t1.cpc_before2week_push)) as cpc_before2week,
array(sum(t1.revenue_before2week_dj),sum(t1.revenue_before2week_bj),sum(t1.revenue_before2week_ss),sum(t1.revenue_before2week_push)) as revenue_before2week,
array(sum(t1.charge_before2week_dj),sum(t1.charge_before2week_bj),sum(t1.charge_before2week_ss),sum(t1.charge_before2week_push)) as charge_before2week,
array(sum(t1.order_num_before2week_dj),sum(t1.order_num_before2week_bj),sum(t1.order_num_before2week_ss),sum(t1.order_num_before2week_push)) as order_num_before2week,
array(sum(t1.mt_butie_before2week_dj),sum(t1.mt_butie_before2week_bj),sum(t1.mt_butie_before2week_ss),sum(t1.mt_butie_before2week_push)) as mt_butie_before2week,
array(sum(t1.impr_before3week_dj),sum(t1.impr_before3week_bj),sum(t1.impr_before3week_ss),sum(t1.impr_before3week_push)) as impr_before3week,
array(sum(t1.cpc_before3week_dj),sum(t1.cpc_before3week_bj),sum(t1.cpc_before3week_ss),sum(t1.cpc_before3week_push)) as cpc_before3week,
array(sum(t1.click_before3week_dj),sum(t1.click_before3week_bj),sum(t1.click_before3week_ss),sum(t1.click_before3week_push)) as click_before3week,
array(sum(t1.order_num_before3week_dj),sum(t1.order_num_before3week_bj),sum(t1.order_num_before3week_ss),sum(t1.order_num_before3week_push)) as order_num_before3week,
array(sum(t1.mt_butie_before3week_dj),sum(t1.mt_butie_before3week_bj),sum(t1.mt_butie_before3week_ss),sum(t1.mt_butie_before3week_push)) as mt_butie_before3week,
array(sum(t1.revenue_before3week_dj),sum(t1.revenue_before3week_bj),sum(t1.revenue_before3week_ss),sum(t1.revenue_before3week_push)) as revenue_before3week,
array(sum(t1.charge_before3week_dj),sum(t1.charge_before3week_bj),sum(t1.charge_before3week_ss),sum(t1.charge_before3week_push)) as charge_before3week,
array(sum(t1.impr_before4week_dj),sum(t1.impr_before4week_bj),sum(t1.impr_before4week_ss),sum(t1.impr_before4week_push)) as impr_before4week,
array(sum(t1.charge_before4week_dj),sum(t1.charge_before4week_bj),sum(t1.charge_before4week_ss),sum(t1.charge_before4week_push)) as charge_before4week,
array(sum(t1.revenue_before4week_dj),sum(t1.revenue_before4week_bj),sum(t1.revenue_before4week_ss),sum(t1.revenue_before4week_push)) as revenue_before4week,
array(sum(t1.mt_butie_before4week_dj),sum(t1.mt_butie_before4week_bj),sum(t1.mt_butie_before4week_ss),sum(t1.mt_butie_before4week_push)) as mt_butie_before4week,
array(sum(t1.cpc_before4week_dj),sum(t1.cpc_before4week_bj),sum(t1.cpc_before4week_ss),sum(t1.cpc_before4week_push)) as cpc_before4week,
array(sum(t1.click_before4week_dj),sum(t1.click_before4week_bj),sum(t1.click_before4week_ss),sum(t1.click_before4week_push)) as click_before4week,
array(sum(t1.order_num_before4week_dj),sum(t1.order_num_before4week_bj),sum(t1.order_num_before4week_ss),sum(t1.order_num_before4week_push)) as order_num_before4week,
array(max(t2.action_dj),max(t2.action_bj),max(t2.action_ss),max(t2.action_push)) as action,
array(sum(t1.charge_dj),sum(t1.charge_bj),sum(t1.charge_ss),sum(t1.charge_push)) as charge,
array(sum(t1.click_dj),sum(t1.click_bj),sum(t1.click_ss),sum(t1.click_push)) as click,
array(sum(t1.reward_dj),sum(t1.reward_bj),sum(t1.reward_ss),sum(t1.reward_push)) as reward,
array(sum(t1.final_charge_dj),sum(t1.final_charge_bj),sum(t1.final_charge_ss),sum(t1.final_charge_push)) as final_charge,
array(sum(t1.budget_dj),sum(t1.budget_bj),sum(t1.budget_ss),sum(t1.budget_push)) as budget,

array(sum(t1.real_csm_dj),sum(t1.real_csm_bj),sum(t1.real_csm_ss),sum(t1.real_csm_push)) as real_csm
 FROM(
        select dt,
               poi_id,
               prod,
if( prod = 1, cast(coalesce(ctr, 0.0) as float) , 0) as ctr_dj,
if( prod = 2, cast(coalesce(ctr, 0.0) as float) , 0) as ctr_bj,
if( prod = 3, cast(coalesce(ctr, 0.0) as float) , 0) as ctr_ss,
if( prod = 4, cast(coalesce(ctr, 0.0) as float) , 0) as ctr_push,
if( prod = 1, cast(coalesce(histctr, 0.0) as float) , 0) as histctr_dj,
if( prod = 2, cast(coalesce(histctr, 0.0) as float) , 0) as histctr_bj,
if( prod = 3, cast(coalesce(histctr, 0.0) as float) , 0) as histctr_ss,
if( prod = 4, cast(coalesce(histctr, 0.0) as float) , 0) as histctr_push,
if( prod = 1, cast(coalesce(final_charge, 0.0) as float) , 0) as final_charge_dj,
if( prod = 2, cast(coalesce(final_charge, 0.0) as float) , 0) as final_charge_bj,
if( prod = 3, cast(coalesce(final_charge, 0.0) as float) , 0) as final_charge_ss,
if( prod = 4, cast(coalesce(final_charge, 0.0) as float) , 0) as final_charge_push,
if( prod = 1, cast(coalesce(aimcpc, 0.0) as float) , 0) as aimcpc_dj,
if( prod = 2, cast(coalesce(aimcpc, 0.0) as float) , 0) as aimcpc_bj,
if( prod = 3, cast(coalesce(aimcpc, 0.0) as float) , 0) as aimcpc_ss,
if( prod = 4, cast(coalesce(aimcpc, 0.0) as float) , 0) as aimcpc_push,
if( prod = 1, cast(coalesce(impr, 0.0) as float) , 0) as impr_dj,
if( prod = 2, cast(coalesce(impr, 0.0) as float) , 0) as impr_bj,
if( prod = 3, cast(coalesce(impr, 0.0) as float) , 0) as impr_ss,
if( prod = 4, cast(coalesce(impr, 0.0) as float) , 0) as impr_push,
if( prod = 1, cast(coalesce(cpc, 0.0) as float) , 0) as cpc_dj,
if( prod = 2, cast(coalesce(cpc, 0.0) as float) , 0) as cpc_bj,
if( prod = 3, cast(coalesce(cpc, 0.0) as float) , 0) as cpc_ss,
if( prod = 4, cast(coalesce(cpc, 0.0) as float) , 0) as cpc_push,
if( prod = 1, cast(coalesce(revenue, 0.0) as float) , 0) as revenue_dj,
if( prod = 2, cast(coalesce(revenue, 0.0) as float) , 0) as revenue_bj,
if( prod = 3, cast(coalesce(revenue, 0.0) as float) , 0) as revenue_ss,
if( prod = 4, cast(coalesce(revenue, 0.0) as float) , 0) as revenue_push,
if( prod = 1, cast(coalesce(charge, 0.0) as float) , 0) as charge_dj,
if( prod = 2, cast(coalesce(charge, 0.0) as float) , 0) as charge_bj,
if( prod = 3, cast(coalesce(charge, 0.0) as float) , 0) as charge_ss,
if( prod = 4, cast(coalesce(charge, 0.0) as float) , 0) as charge_push,
if( prod = 1, cast(coalesce(click, 0.0) as float) , 0) as click_dj,
if( prod = 2, cast(coalesce(click, 0.0) as float) , 0) as click_bj,
if( prod = 3, cast(coalesce(click, 0.0) as float) , 0) as click_ss,
if( prod = 4, cast(coalesce(click, 0.0) as float) , 0) as click_push,
if( prod = 1, cast(coalesce(order_num, 0.0) as float) , 0) as order_num_dj,
if( prod = 2, cast(coalesce(order_num, 0.0) as float) , 0) as order_num_bj,
if( prod = 3, cast(coalesce(order_num, 0.0) as float) , 0) as order_num_ss,
if( prod = 4, cast(coalesce(order_num, 0.0) as float) , 0) as order_num_push,
if( prod = 1, cast(coalesce(mt_butie, 0.0) as float) , 0) as mt_butie_dj,
if( prod = 2, cast(coalesce(mt_butie, 0.0) as float) , 0) as mt_butie_bj,
if( prod = 3, cast(coalesce(mt_butie, 0.0) as float) , 0) as mt_butie_ss,
if( prod = 4, cast(coalesce(mt_butie, 0.0) as float) , 0) as mt_butie_push,
if( prod = 1, cast(coalesce(budget, 0.0) as float) , 0) as budget_dj,
if( prod = 2, cast(coalesce(budget, 0.0) as float) , 0) as budget_bj,
if( prod = 3, cast(coalesce(budget, 0.0) as float) , 0) as budget_ss,
if( prod = 4, cast(coalesce(budget, 0.0) as float) , 0) as budget_push,
if( prod = 1, cast(coalesce(impr_before1week, 0.0) as float) , 0) as impr_before1week_dj,
if( prod = 2, cast(coalesce(impr_before1week, 0.0) as float) , 0) as impr_before1week_bj,
if( prod = 3, cast(coalesce(impr_before1week, 0.0) as float) , 0) as impr_before1week_ss,
if( prod = 4, cast(coalesce(impr_before1week, 0.0) as float) , 0) as impr_before1week_push,
if( prod = 1, cast(coalesce(charge_before1week, 0.0) as float) , 0) as charge_before1week_dj,
if( prod = 2, cast(coalesce(charge_before1week, 0.0) as float) , 0) as charge_before1week_bj,
if( prod = 3, cast(coalesce(charge_before1week, 0.0) as float) , 0) as charge_before1week_ss,
if( prod = 4, cast(coalesce(charge_before1week, 0.0) as float) , 0) as charge_before1week_push,
if( prod = 1, cast(coalesce(click_before1week, 0.0) as float) , 0) as click_before1week_dj,
if( prod = 2, cast(coalesce(click_before1week, 0.0) as float) , 0) as click_before1week_bj,
if( prod = 3, cast(coalesce(click_before1week, 0.0) as float) , 0) as click_before1week_ss,
if( prod = 4, cast(coalesce(click_before1week, 0.0) as float) , 0) as click_before1week_push,
if( prod = 1, cast(coalesce(cpc_before1week, 0.0) as float) , 0) as cpc_before1week_dj,
if( prod = 2, cast(coalesce(cpc_before1week, 0.0) as float) , 0) as cpc_before1week_bj,
if( prod = 3, cast(coalesce(cpc_before1week, 0.0) as float) , 0) as cpc_before1week_ss,
if( prod = 4, cast(coalesce(cpc_before1week, 0.0) as float) , 0) as cpc_before1week_push,
if( prod = 1, cast(coalesce(order_num_before1week, 0.0) as float) , 0) as order_num_before1week_dj,
if( prod = 2, cast(coalesce(order_num_before1week, 0.0) as float) , 0) as order_num_before1week_bj,
if( prod = 3, cast(coalesce(order_num_before1week, 0.0) as float) , 0) as order_num_before1week_ss,
if( prod = 4, cast(coalesce(order_num_before1week, 0.0) as float) , 0) as order_num_before1week_push,
if( prod = 1, cast(coalesce(revenue_before1week, 0.0) as float) , 0) as revenue_before1week_dj,
if( prod = 2, cast(coalesce(revenue_before1week, 0.0) as float) , 0) as revenue_before1week_bj,
if( prod = 3, cast(coalesce(revenue_before1week, 0.0) as float) , 0) as revenue_before1week_ss,
if( prod = 4, cast(coalesce(revenue_before1week, 0.0) as float) , 0) as revenue_before1week_push,
if( prod = 1, cast(coalesce(mt_butie_before1week, 0.0) as float) , 0) as mt_butie_before1week_dj,
if( prod = 2, cast(coalesce(mt_butie_before1week, 0.0) as float) , 0) as mt_butie_before1week_bj,
if( prod = 3, cast(coalesce(mt_butie_before1week, 0.0) as float) , 0) as mt_butie_before1week_ss,
if( prod = 4, cast(coalesce(mt_butie_before1week, 0.0) as float) , 0) as mt_butie_before1week_push,
if( prod = 1, cast(coalesce(impr_before2week, 0.0) as float) , 0) as impr_before2week_dj,
if( prod = 2, cast(coalesce(impr_before2week, 0.0) as float) , 0) as impr_before2week_bj,
if( prod = 3, cast(coalesce(impr_before2week, 0.0) as float) , 0) as impr_before2week_ss,
if( prod = 4, cast(coalesce(impr_before2week, 0.0) as float) , 0) as impr_before2week_push,
if( prod = 1, cast(coalesce(charge_before2week, 0.0) as float) , 0) as charge_before2week_dj,
if( prod = 2, cast(coalesce(charge_before2week, 0.0) as float) , 0) as charge_before2week_bj,
if( prod = 3, cast(coalesce(charge_before2week, 0.0) as float) , 0) as charge_before2week_ss,
if( prod = 4, cast(coalesce(charge_before2week, 0.0) as float) , 0) as charge_before2week_push,
if( prod = 1, cast(coalesce(click_before2week, 0.0) as float) , 0) as click_before2week_dj,
if( prod = 2, cast(coalesce(click_before2week, 0.0) as float) , 0) as click_before2week_bj,
if( prod = 3, cast(coalesce(click_before2week, 0.0) as float) , 0) as click_before2week_ss,
if( prod = 4, cast(coalesce(click_before2week, 0.0) as float) , 0) as click_before2week_push,
if( prod = 1, cast(coalesce(cpc_before2week, 0.0) as float) , 0) as cpc_before2week_dj,
if( prod = 2, cast(coalesce(cpc_before2week, 0.0) as float) , 0) as cpc_before2week_bj,
if( prod = 3, cast(coalesce(cpc_before2week, 0.0) as float) , 0) as cpc_before2week_ss,
if( prod = 4, cast(coalesce(cpc_before2week, 0.0) as float) , 0) as cpc_before2week_push,
if( prod = 1, cast(coalesce(order_num_before2week, 0.0) as float) , 0) as order_num_before2week_dj,
if( prod = 2, cast(coalesce(order_num_before2week, 0.0) as float) , 0) as order_num_before2week_bj,
if( prod = 3, cast(coalesce(order_num_before2week, 0.0) as float) , 0) as order_num_before2week_ss,
if( prod = 4, cast(coalesce(order_num_before2week, 0.0) as float) , 0) as order_num_before2week_push,
if( prod = 1, cast(coalesce(revenue_before2week, 0.0) as float) , 0) as revenue_before2week_dj,
if( prod = 2, cast(coalesce(revenue_before2week, 0.0) as float) , 0) as revenue_before2week_bj,
if( prod = 3, cast(coalesce(revenue_before2week, 0.0) as float) , 0) as revenue_before2week_ss,
if( prod = 4, cast(coalesce(revenue_before2week, 0.0) as float) , 0) as revenue_before2week_push,
if( prod = 1, cast(coalesce(mt_butie_before2week, 0.0) as float) , 0) as mt_butie_before2week_dj,
if( prod = 2, cast(coalesce(mt_butie_before2week, 0.0) as float) , 0) as mt_butie_before2week_bj,
if( prod = 3, cast(coalesce(mt_butie_before2week, 0.0) as float) , 0) as mt_butie_before2week_ss,
if( prod = 4, cast(coalesce(mt_butie_before2week, 0.0) as float) , 0) as mt_butie_before2week_push,
if( prod = 1, cast(coalesce(impr_before3week, 0.0) as float) , 0) as impr_before3week_dj,
if( prod = 2, cast(coalesce(impr_before3week, 0.0) as float) , 0) as impr_before3week_bj,
if( prod = 3, cast(coalesce(impr_before3week, 0.0) as float) , 0) as impr_before3week_ss,
if( prod = 4, cast(coalesce(impr_before3week, 0.0) as float) , 0) as impr_before3week_push,
if( prod = 1, cast(coalesce(charge_before3week, 0.0) as float) , 0) as charge_before3week_dj,
if( prod = 2, cast(coalesce(charge_before3week, 0.0) as float) , 0) as charge_before3week_bj,
if( prod = 3, cast(coalesce(charge_before3week, 0.0) as float) , 0) as charge_before3week_ss,
if( prod = 4, cast(coalesce(charge_before3week, 0.0) as float) , 0) as charge_before3week_push,
if( prod = 1, cast(coalesce(click_before3week, 0.0) as float) , 0) as click_before3week_dj,
if( prod = 2, cast(coalesce(click_before3week, 0.0) as float) , 0) as click_before3week_bj,
if( prod = 3, cast(coalesce(click_before3week, 0.0) as float) , 0) as click_before3week_ss,
if( prod = 4, cast(coalesce(click_before3week, 0.0) as float) , 0) as click_before3week_push,
if( prod = 1, cast(coalesce(cpc_before3week, 0.0) as float) , 0) as cpc_before3week_dj,
if( prod = 2, cast(coalesce(cpc_before3week, 0.0) as float) , 0) as cpc_before3week_bj,
if( prod = 3, cast(coalesce(cpc_before3week, 0.0) as float) , 0) as cpc_before3week_ss,
if( prod = 4, cast(coalesce(cpc_before3week, 0.0) as float) , 0) as cpc_before3week_push,
if( prod = 1, cast(coalesce(order_num_before3week, 0.0) as float) , 0) as order_num_before3week_dj,
if( prod = 2, cast(coalesce(order_num_before3week, 0.0) as float) , 0) as order_num_before3week_bj,
if( prod = 3, cast(coalesce(order_num_before3week, 0.0) as float) , 0) as order_num_before3week_ss,
if( prod = 4, cast(coalesce(order_num_before3week, 0.0) as float) , 0) as order_num_before3week_push,
if( prod = 1, cast(coalesce(revenue_before3week, 0.0) as float) , 0) as revenue_before3week_dj,
if( prod = 2, cast(coalesce(revenue_before3week, 0.0) as float) , 0) as revenue_before3week_bj,
if( prod = 3, cast(coalesce(revenue_before3week, 0.0) as float) , 0) as revenue_before3week_ss,
if( prod = 4, cast(coalesce(revenue_before3week, 0.0) as float) , 0) as revenue_before3week_push,
if( prod = 1, cast(coalesce(mt_butie_before3week, 0.0) as float) , 0) as mt_butie_before3week_dj,
if( prod = 2, cast(coalesce(mt_butie_before3week, 0.0) as float) , 0) as mt_butie_before3week_bj,
if( prod = 3, cast(coalesce(mt_butie_before3week, 0.0) as float) , 0) as mt_butie_before3week_ss,
if( prod = 4, cast(coalesce(mt_butie_before3week, 0.0) as float) , 0) as mt_butie_before3week_push,
if( prod = 1, cast(coalesce(impr_before4week, 0.0) as float) , 0) as impr_before4week_dj,
if( prod = 2, cast(coalesce(impr_before4week, 0.0) as float) , 0) as impr_before4week_bj,
if( prod = 3, cast(coalesce(impr_before4week, 0.0) as float) , 0) as impr_before4week_ss,
if( prod = 4, cast(coalesce(impr_before4week, 0.0) as float) , 0) as impr_before4week_push,
if( prod = 1, cast(coalesce(charge_before4week, 0.0) as float) , 0) as charge_before4week_dj,
if( prod = 2, cast(coalesce(charge_before4week, 0.0) as float) , 0) as charge_before4week_bj,
if( prod = 3, cast(coalesce(charge_before4week, 0.0) as float) , 0) as charge_before4week_ss,
if( prod = 4, cast(coalesce(charge_before4week, 0.0) as float) , 0) as charge_before4week_push,
if( prod = 1, cast(coalesce(click_before4week, 0.0) as float) , 0) as click_before4week_dj,
if( prod = 2, cast(coalesce(click_before4week, 0.0) as float) , 0) as click_before4week_bj,
if( prod = 3, cast(coalesce(click_before4week, 0.0) as float) , 0) as click_before4week_ss,
if( prod = 4, cast(coalesce(click_before4week, 0.0) as float) , 0) as click_before4week_push,
if( prod = 1, cast(coalesce(cpc_before4week, 0.0) as float) , 0) as cpc_before4week_dj,
if( prod = 2, cast(coalesce(cpc_before4week, 0.0) as float) , 0) as cpc_before4week_bj,
if( prod = 3, cast(coalesce(cpc_before4week, 0.0) as float) , 0) as cpc_before4week_ss,
if( prod = 4, cast(coalesce(cpc_before4week, 0.0) as float) , 0) as cpc_before4week_push,
if( prod = 1, cast(coalesce(order_num_before4week, 0.0) as float) , 0) as order_num_before4week_dj,
if( prod = 2, cast(coalesce(order_num_before4week, 0.0) as float) , 0) as order_num_before4week_bj,
if( prod = 3, cast(coalesce(order_num_before4week, 0.0) as float) , 0) as order_num_before4week_ss,
if( prod = 4, cast(coalesce(order_num_before4week, 0.0) as float) , 0) as order_num_before4week_push,
if( prod = 1, cast(coalesce(revenue_before4week, 0.0) as float) , 0) as revenue_before4week_dj,
if( prod = 2, cast(coalesce(revenue_before4week, 0.0) as float) , 0) as revenue_before4week_bj,
if( prod = 3, cast(coalesce(revenue_before4week, 0.0) as float) , 0) as revenue_before4week_ss,
if( prod = 4, cast(coalesce(revenue_before4week, 0.0) as float) , 0) as revenue_before4week_push,
if( prod = 1, cast(coalesce(mt_butie_before4week, 0.0) as float) , 0) as mt_butie_before4week_dj,
if( prod = 2, cast(coalesce(mt_butie_before4week, 0.0) as float) , 0) as mt_butie_before4week_bj,
if( prod = 3, cast(coalesce(mt_butie_before4week, 0.0) as float) , 0) as mt_butie_before4week_ss,
if( prod = 4, cast(coalesce(mt_butie_before4week, 0.0) as float) , 0) as mt_butie_before4week_push,

if( prod = 1,  cast(coalesce(click_nums, 0.0) as float  ) , 0) as reward_dj,
if( prod = 2,  cast(coalesce(click_nums, 0.0) as float  ) , 0) as reward_bj,
if( prod = 3,  cast(coalesce(click_nums, 0.0) as float  ) , 0) as reward_ss,
if( prod = 4,  cast(coalesce(click_nums, 0.0) as float  ) , 0) as reward_push,

if( prod = 1, cast(coalesce(real_csm, 0.0) as float) , 0) as real_csm_dj,
if( prod = 2, cast(coalesce(real_csm, 0.0) as float) , 0) as real_csm_bj,
if( prod = 3, cast(coalesce(real_csm, 0.0) as float) , 0) as real_csm_ss,
if( prod = 4, cast(coalesce(real_csm, 0.0) as float) , 0) as real_csm_push

               
          from ${processTable_high}
       )t1
  left join(
        SELECT poi_id,
       dt,
  CASE WHEN jifei_bj/budget >= 0.0 AND jifei_dj/budget < 0.05 THEN 0
 WHEN jifei_bj/budget >= 0.05 AND jifei_dj/budget < 0.1 THEN 1
 WHEN jifei_bj/budget >= 0.1 AND jifei_dj/budget < 0.15 THEN 2
 WHEN jifei_bj/budget >= 0.15 AND jifei_dj/budget < 0.2 THEN 3
 WHEN jifei_bj/budget >= 0.2 AND jifei_dj/budget < 0.25 THEN 4
 WHEN jifei_bj/budget >= 0.25 AND jifei_dj/budget < 0.3 THEN 5
 WHEN jifei_bj/budget >= 0.3 AND jifei_dj/budget < 0.35 THEN 6
 WHEN jifei_bj/budget >= 0.35 AND jifei_dj/budget < 0.4 THEN 7
 WHEN jifei_bj/budget >= 0.4 AND jifei_dj/budget < 0.45 THEN 8
 WHEN jifei_bj/budget >= 0.45 AND jifei_dj/budget < 0.5 THEN 9
 WHEN jifei_bj/budget >= 0.5 AND jifei_dj/budget < 0.55 THEN 10
 WHEN jifei_bj/budget >= 0.55 AND jifei_dj/budget < 0.6 THEN 11
 WHEN jifei_bj/budget >= 0.6 AND jifei_dj/budget < 0.65 THEN 12
 WHEN jifei_bj/budget >= 0.65 AND jifei_dj/budget < 0.7 THEN 13
 WHEN jifei_bj/budget >= 0.7 AND jifei_dj/budget < 0.75 THEN 14
 WHEN jifei_bj/budget >= 0.75 AND jifei_dj/budget < 0.8 THEN 15
 WHEN jifei_bj/budget >= 0.8 AND jifei_dj/budget < 0.85 THEN 16
 WHEN jifei_bj/budget >= 0.85 AND jifei_dj/budget < 0.9 THEN 17
 WHEN jifei_bj/budget >= 0.9 AND jifei_dj/budget < 0.95 THEN 18
 WHEN jifei_bj/budget >= 0.95 AND jifei_dj/budget < 1.0 THEN 19
  ELSE 20
END as action_dj,
CASE WHEN jifei_dj/budget >= 0.0 AND jifei_dj/budget < 0.05 THEN 0
 WHEN jifei_dj/budget >= 0.05 AND jifei_dj/budget < 0.1 THEN 1
 WHEN jifei_dj/budget >= 0.1 AND jifei_dj/budget < 0.15 THEN 2
 WHEN jifei_dj/budget >= 0.15 AND jifei_dj/budget < 0.2 THEN 3
 WHEN jifei_dj/budget >= 0.2 AND jifei_dj/budget < 0.25 THEN 4
 WHEN jifei_dj/budget >= 0.25 AND jifei_dj/budget < 0.3 THEN 5
 WHEN jifei_dj/budget >= 0.3 AND jifei_dj/budget < 0.35 THEN 6
 WHEN jifei_dj/budget >= 0.35 AND jifei_dj/budget < 0.4 THEN 7
 WHEN jifei_dj/budget >= 0.4 AND jifei_dj/budget < 0.45 THEN 8
 WHEN jifei_dj/budget >= 0.45 AND jifei_dj/budget < 0.5 THEN 9
 WHEN jifei_dj/budget >= 0.5 AND jifei_dj/budget < 0.55 THEN 10
 WHEN jifei_dj/budget >= 0.55 AND jifei_dj/budget < 0.6 THEN 11
 WHEN jifei_dj/budget >= 0.6 AND jifei_dj/budget < 0.65 THEN 12
 WHEN jifei_dj/budget >= 0.65 AND jifei_dj/budget < 0.7 THEN 13
 WHEN jifei_dj/budget >= 0.7 AND jifei_dj/budget < 0.75 THEN 14
 WHEN jifei_dj/budget >= 0.75 AND jifei_dj/budget < 0.8 THEN 15
 WHEN jifei_dj/budget >= 0.8 AND jifei_dj/budget < 0.85 THEN 16
 WHEN jifei_dj/budget >= 0.85 AND jifei_dj/budget < 0.9 THEN 17
 WHEN jifei_dj/budget >= 0.9 AND jifei_dj/budget < 0.95 THEN 18
 WHEN jifei_dj/budget >= 0.95 AND jifei_dj/budget < 1.0 THEN 19
   ELSE 20
END as action_bj,
 CASE WHEN jifei_ss/budget >= 0.0 AND jifei_dj/budget < 0.05 THEN 0
 WHEN jifei_ss/budget >= 0.05 AND jifei_dj/budget < 0.1 THEN 1
 WHEN jifei_ss/budget >= 0.1 AND jifei_dj/budget < 0.15 THEN 2
 WHEN jifei_ss/budget >= 0.15 AND jifei_dj/budget < 0.2 THEN 3
 WHEN jifei_ss/budget >= 0.2 AND jifei_dj/budget < 0.25 THEN 4
 WHEN jifei_ss/budget >= 0.25 AND jifei_dj/budget < 0.3 THEN 5
 WHEN jifei_ss/budget >= 0.3 AND jifei_dj/budget < 0.35 THEN 6
 WHEN jifei_ss/budget >= 0.35 AND jifei_dj/budget < 0.4 THEN 7
 WHEN jifei_ss/budget >= 0.4 AND jifei_dj/budget < 0.45 THEN 8
 WHEN jifei_ss/budget >= 0.45 AND jifei_dj/budget < 0.5 THEN 9
 WHEN jifei_ss/budget >= 0.5 AND jifei_dj/budget < 0.55 THEN 10
 WHEN jifei_ss/budget >= 0.55 AND jifei_dj/budget < 0.6 THEN 11
 WHEN jifei_ss/budget >= 0.6 AND jifei_dj/budget < 0.65 THEN 12
 WHEN jifei_ss/budget >= 0.65 AND jifei_dj/budget < 0.7 THEN 13
 WHEN jifei_ss/budget >= 0.7 AND jifei_dj/budget < 0.75 THEN 14
 WHEN jifei_ss/budget >= 0.75 AND jifei_dj/budget < 0.8 THEN 15
 WHEN jifei_ss/budget >= 0.8 AND jifei_dj/budget < 0.85 THEN 16
 WHEN jifei_ss/budget >= 0.85 AND jifei_dj/budget < 0.9 THEN 17
 WHEN jifei_ss/budget >= 0.9 AND jifei_dj/budget < 0.95 THEN 18
 WHEN jifei_ss/budget >= 0.95 AND jifei_dj/budget < 1.0 THEN 19
   ELSE 20
        END as action_ss,
       
CASE WHEN jifei_push/budget >= 0.0 AND jifei_dj/budget < 0.05 THEN 0
 WHEN jifei_push/budget >= 0.05 AND jifei_dj/budget < 0.1 THEN 1
 WHEN jifei_push/budget >= 0.1 AND jifei_dj/budget < 0.15 THEN 2
 WHEN jifei_push/budget >= 0.15 AND jifei_dj/budget < 0.2 THEN 3
 WHEN jifei_push/budget >= 0.2 AND jifei_dj/budget < 0.25 THEN 4
 WHEN jifei_push/budget >= 0.25 AND jifei_dj/budget < 0.3 THEN 5
 WHEN jifei_push/budget >= 0.3 AND jifei_dj/budget < 0.35 THEN 6
 WHEN jifei_push/budget >= 0.35 AND jifei_dj/budget < 0.4 THEN 7
 WHEN jifei_push/budget >= 0.4 AND jifei_dj/budget < 0.45 THEN 8
 WHEN jifei_push/budget >= 0.45 AND jifei_dj/budget < 0.5 THEN 9
 WHEN jifei_push/budget >= 0.5 AND jifei_dj/budget < 0.55 THEN 10
 WHEN jifei_push/budget >= 0.55 AND jifei_dj/budget < 0.6 THEN 11
 WHEN jifei_push/budget >= 0.6 AND jifei_dj/budget < 0.65 THEN 12
 WHEN jifei_push/budget >= 0.65 AND jifei_dj/budget < 0.7 THEN 13
 WHEN jifei_push/budget >= 0.7 AND jifei_dj/budget < 0.75 THEN 14
 WHEN jifei_push/budget >= 0.75 AND jifei_dj/budget < 0.8 THEN 15
 WHEN jifei_push/budget >= 0.8 AND jifei_dj/budget < 0.85 THEN 16
 WHEN jifei_push/budget >= 0.85 AND jifei_dj/budget < 0.9 THEN 17
 WHEN jifei_push/budget >= 0.9 AND jifei_dj/budget < 0.95 THEN 18
 WHEN jifei_push/budget >= 0.95 AND jifei_dj/budget < 1.0 THEN 19
   ELSE 20
        END as action_push
  FROM (
        select poi_id,
               dt,
               max(budget) as budget,
               max(jifei_dj) as jifei_dj,
               max(jifei_bj) as jifei_bj,
               max(jifei_ss) as jifei_ss,
               max(jifei_push) as jifei_push from(
                SELECT poi_id,
                       dt,
                       budget,
                       if(prod=1.0, real_csm, 0) as jifei_dj,
                       if(prod=2.0, real_csm, 0) as jifei_bj,
                       if(prod=3.0, real_csm, 0) as jifei_ss,
                       if(prod=4.0, real_csm, 0) as jifei_push
                  FROM ${processTable_high}
               )
         GROUP BY poi_id,
                  dt
       )
     )t2
    ON t1.dt = t2.dt
   AND t1.poi_id = t2.poi_id
   group by t1.dt,t1.poi_id
""")


2023-01-04 19:00:45
执行 
 
SELECT t1.dt,
t1.poi_id,
array(sum(t1.ctr_dj),sum(t1.ctr_bj),sum(t1.ctr_ss),sum(t1.ctr_push)) as ctr,
array(sum(t1.histctr_dj),sum(t1.histctr_bj),sum(t1.histctr_ss),sum(t1.histctr_push)) as histctr,
array(sum(t1.aimcpc_dj),sum(t1.aimcpc_bj),sum(t1.aimcpc_ss),sum(t1.aimcpc_push)) as aimcpc,
array(sum(t1.impr_dj),sum(t1.impr_bj),sum(t1.impr_ss),sum(t1.impr_push)) as impr,
array(sum(t1.cpc_dj),sum(t1.cpc_bj),sum(t1.cpc_ss),sum(t1.cpc_push)) as cpc,
array(sum(t1.revenue_dj),sum(t1.revenue_bj),sum(t1.revenue_ss),sum(t1.revenue_push)) as revenue,
array(sum(t1.order_num_dj),sum(t1.order_num_bj),sum(t1.order_num_ss),sum(t1.order_num_push)) as order_num,
array(sum(t1.mt_butie_dj),sum(t1.mt_butie_bj),sum(t1.mt_butie_ss),sum(t1.mt_butie_push)) as mt_butie,
array(sum(t1.impr_before1week_dj),sum(t1.impr_before1week_bj),sum(t1.impr_before1week_ss),sum(t1.impr_before1week_push)) as impr_before1week,
array(sum(t1.click_before1week_dj),sum(t1.click_before1week_bj),sum(t1.click

run at ThreadPoolExecutor.java:1142

200 / 200

[Stage 25:=====================================================>(199 + 1) / 200]

run at ThreadPoolExecutor.java:1142

...

[Stage 27:=====================================================>(199 + 1) / 200]

sql at cmd2.sc:66

200 / 200

Moved: 'viewfs://hadoop-meituan/user/hive/warehouse/tmp.db/afo_rl_1672830045261' to trash at: hdfs://dfsrouter/user/hadoop-hmart-waimaiad/.Trash/Current
2023-01-04 19:07:15


tableNameAfterTrajectory_high: String = "tmp.AFO_RL_1672830045261"

In [20]:
// var tableNameAfterTrajectory_high = "tmp.AFO_RL_1665933624473"
var tableNameAfterTrajectory = tableNameAfterTrajectory_high

tableNameAfterTrajectory: String = "tmp.AFO_RL_1672830045261"

In [21]:
// // var tableNameAfterTrajectory = "mart_waimaiad.AFO_RL_1664328251754"
// var tmpTrainTable = createTableFromSQL(s"""
// select *
// from ${tableNameAfterTrajectory}
// where dt>=${fp_log_date_start}
// and dt<= ${fp_log_date_end}
// """)
//因为hive直读当前需要分区表，所以创建一个分区表
var finalTrainTable = createPTTableFromTableWithSpace(tableNameAfterTrajectory,"mart_waimaiad")


2023-01-04 19:07:15
执行 
 select * from tmp.AFO_RL_1672830045261
2023-01-04 19:07:15
2023-01-04 19:07:15
执行 
 drop table if exists mart_waimaiad.AFO_RL_1672830435811
2023-01-04 19:07:15
2023-01-04 19:07:15
执行 
 create table if not exists mart_waimaiad.AFO_RL_1672830435811(dt String,poi_id String,ctr Array<Double>,histctr Array<Double>,aimcpc Array<Double>,impr Array<Double>,cpc Array<Double>,revenue Array<Double>,order_num Array<Double>,mt_butie Array<Double>,impr_before1week Array<Double>,click_before1week Array<Double>,cpc_before1week Array<Double>,order_num_before1week Array<Double>,revenue_before1week Array<Double>,mt_butie_before1week Array<Double>,charge_before1week Array<Double>,impr_before2week Array<Double>,click_before2week Array<Double>,cpc_before2week Array<Double>,revenue_before2week Array<Double>,charge_before2week Array<Double>,order_num_before2week Array<Double>,mt_butie_before2week Array<Double>,impr_before3week Array<Double>,cpc_before3week Array<Double>,click_before3w

sql at cmd2.sc:236

200 / 200

finalTrainTable: String = "mart_waimaiad.AFO_RL_1672830435811"

In [22]:
tableNameAfterTrajectory
finalTrainTable
getDate(-1,"today")

res21_0: String = "tmp.AFO_RL_1672830045261"
res21_1: String = "mart_waimaiad.AFO_RL_1672830435811"
res21_2: String = "20230103"

In [13]:
showTopNFromSQL(s"""
select count(*) from ${finalTrainTable}
""",1)

2023-01-04 18:38:59
2023-01-04 18:38:59
执行 
 
select count(*) from mart_waimaiad.AFO_RL_1672828332426

2023-01-04 18:38:59


show at cmd2.sc:96

200 / 200

[Stage 19:=====================================================>(199 + 1) / 200]

show at cmd2.sc:96

1 / 1

+--------+
|count(1)|
+--------+
|   64582|
+--------+

2023-01-04 18:39:04


In [18]:
showTopNFromSQL(s"""
select * from ${finalTrainTable}
""",10)

2023-01-04 18:59:35
2023-01-04 18:59:35
执行 
 
select * from mart_waimaiad.AFO_RL_1672828332426

2023-01-04 18:59:35


show at cmd2.sc:96

1 / 1

+--------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------------+--------------------+--------------------+--------------------+--------------------+---------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+
|      dt|  poi_id|                 ctr|             histctr|       

## 